In [1652]:
import pandas as pd
import numpy as np
import os
import os.path as path

In [1653]:
data_path = "./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv"

In [1654]:
# csv_name_list = os.listdir(data_path)

In [1655]:
# print(csv_name_list)

In [1656]:
# for csv_name in csv_name_list:
#     print("===================={}===================".format(csv_name))
#     df = pd.read_csv(path.join(data_path, csv_name))
#     print(df.describe())

In [1657]:
def print_isna(df):
    print(pd.isna(df).sum())

# 학습 데이터 셋 만들기

# INPUT

> 소득
    - INC || INCOME, HOUSE_INCOME <========== tn_traveller_master_여행객 Master_C 

    특이사항 - 소득 값이 두가지 존재

    INCOMDE사용으로 일단 대체
    
> 미션
    - MIS || TRAVEL_MISSION(==TRAVEL_PURPOSE) <========== tn_travel_여행_C

    특이사항 - 여행 미션내 값들이 여러개 존재

> 여행동반자관계
    - TCR || REL_CD <========== tn_companion_info_동반자정보_C

> 숙소유형 
    - HTY || LODGING_TYPE_CD <========= tn_visit_area_info_방문지정보_C


> 성별
    - GEN || GENDER  <========== tn_traveller_master_여행객 Master_C

    특이사항 남, 여로 적혀 있음

    해당 갑은 남 : 1, 여 : 0으로 변경

> 여행동기
    - TMT || TRAVEL_MOTiVE_1 ~ 3 <========== tn_traveller_master_여행객 Master_C

    해당 항목이 1~3번 존재

    3가지 중복 없이 입력

> 여행스타일
    - STY || TRAVEL_STYL_1 ~ 8  <========== tn_traveller_master_여행객 Master_C

    해당 항목이 1~8번이 존재

    => 1개만 받을 예정
    학습 시 max값으로 대체

> 동반자연령대
    - AGE || COMPANION_AGE_GRP <============ tn_companion_info_동반자정보_C
    

#################################### 여기 까지는 후에 타 서버에서 받아서 예측시 입력해 줄 데이터 #####################################



#################################################################################################################################

# OUTPUT
 

> (DGSTFN + REVISIT_INTENTOIN + RCMDTN_INTENTION) / 3 =======>>>>>>>>> 실제 출력
    
    모두 tn_visit_area_info_방문지정보_C 에 존재

    '()' 안의 더해지는 값들은 각 값의 범위가 1~

> 여행지 만족도

    train 간에는 모델은 입력된 여행 데이터를 바탕으로 각 여행지의 점수를 맞추는 작업을 수행한다.
        
        출력한 점수 중 상위 k개의 장소를 선정 해당 장소 중에 정답이 존재한다면
    
    predict의 경우 모든 여행지에 대해서 점수를 매기는 작업 수행 -> softmax를 통해서 표준화하고 내림차순 정렬



> 여행 ID  ======>>>>>> 필요 X
    - || TRAVEL_ID <======= tn_traveller_master_여행객 Master_C

> 방문지 주소 # 전처리 후 리스트해서 전달
    - - || LOTNO_ADDR <====== tn_visit_area_info_방문지정보_C

    몇 항목만 사용
        VIS
            1:자연관광지
            2:역사/유적/종교 시설
            3:문화 시설
            4:상업지구
            5:레저
            6:테마시설
            7:산책로, 둘레길 등
            8:지역 축제/행사
            13: 체험 활동 관광지

            보류
                10:상점
                11:식당/카페
                12:기타
                24:숙소

            제거
                21:집
                22:친구/친구집
                23:사무실
                

2가지 경우가 존재

1. travel id를 확률을 맞추게하고

    2. 확률 분포를 이용 해당 분포 순으로 해당 id가 방문한 지역 전달

2. 방문지 수에 맞게 데이터를 확장한다.

    2. 이 경우 추가적인 데이터를 심어주는게 좋을 듯하다 예를 들어 순서 방문 순서 정보나 재방문 점수 같은 정보를 추가해서 유저 정보 + 다른 추가 정보 이용 학습


In [1658]:
input_data_csv_name_list = ["tn_traveller_master_여행객 Master_C", "tn_travel_여행_C", "tn_companion_info_동반자정보_C", "tn_visit_area_info_방문지정보_C"]

In [1659]:
output_data_csv_name_list = ["tn_travel_여행_C","tn_visit_area_info_방문지정보_C"]

In [1660]:
data_path = "./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv"
extender = '.csv'

index_dict = {'TRAVEL_ID':['1']}
# input_dict = {'여행객':[],'여행':[],'동반자정보':[],'방문지정보':[]}
# 방문지 정보는 output에서 같이 처리
input_dict = {'여행객':[],'여행':[],'동반자정보':[]}
output_dict = {'TRAVELER_ID':[],'방문지정보':[]}
print(index_dict)
print(input_dict)
print(output_dict)
print(data_path)

{'TRAVEL_ID': ['1']}
{'여행객': [], '여행': [], '동반자정보': []}
{'TRAVELER_ID': [], '방문지정보': []}
./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv


In [1661]:
tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[0]+extender))

index_list = tmp_df['TRAVELER_ID'].values
index_dict['TRAVEL_ID'] = index_list
print(index_dict)
# print(tmp_df['TRAVELER_ID'].values)

{'TRAVEL_ID': array(['c003867', 'c000679', 'c013674', ..., 'c008277', 'c012053',
       'c007519'], dtype=object)}


In [1662]:
get_list = ['TRAVELER_ID','INCOME', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1','TRAVEL_MOTIVE_2','TRAVEL_MOTIVE_3', 
            'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[0]+extender))
tmp_value = tmp_df['TRAVELER_ID'].values

tmp_value = [ s[0]+"_"+s for s in tmp_value ]
tmp_df['TRAVEL_ID'] = tmp_value
tmp_df = tmp_df.drop('TRAVELER_ID', axis='columns')

tmp_df = tmp_df.set_index("TRAVEL_ID")

input_dict['여행객'] = tmp_df[get_list[1:]]
print(input_dict['여행객'].columns)
print(input_dict['여행객'].head())


Index(['INCOME', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8'],
      dtype='object')
           INCOME  HOUSE_INCOME  TRAVEL_MOTIVE_1  TRAVEL_MOTIVE_2  \
TRAVEL_ID                                                           
c_c003867       4           NaN              1.0              7.0   
c_c000679       4           NaN              5.0              3.0   
c_c013674       4           6.0              1.0              8.0   
c_c010826       1           1.0              7.0              2.0   
c_c012020       3           7.0              3.0              1.0   

           TRAVEL_MOTIVE_3  TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  \
TRAVEL_ID                                                                 
c_c003867              2.0              2              1              2   
c_c000679            

In [1663]:
get_list = ['TRAVEL_ID', 'TRAVEL_MISSION']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[1]+extender))
tmp_df = tmp_df.set_index(get_list[0])

input_dict['여행'] = tmp_df[get_list[1:]]
print_isna(input_dict['여행'])

TRAVEL_MISSION    0
dtype: int64


In [1664]:
get_list = ['TRAVEL_ID', 'REL_CD', 'COMPANION_AGE_GRP']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[2]+extender))
tmp_df = tmp_df.set_index(get_list[0])
input_dict['동반자정보'] = tmp_df[get_list[1:]]
print_isna(input_dict['동반자정보'])

REL_CD               0
COMPANION_AGE_GRP    0
dtype: int64


In [1665]:
# get_list = ['TRAVEL_ID', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD']

# tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[3]+extender))
# tmp_df = tmp_df.set_index(get_list[0])
# input_dict['방문지정보'] = tmp_df[get_list[1:]]
# # print(tmp_df)

"""output_dict의 방문지정보에서 같이 처리"""

'output_dict의 방문지정보에서 같이 처리'

In [1666]:
get_list = ['TRAVEL_ID','TRAVELER_ID']

tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[0]+extender))
tmp_df = tmp_df.set_index(get_list[0])
output_dict['TRAVELER_ID'] = tmp_df[get_list[1:]]
# print(output_dict['TRAVELER_ID'])
print_isna(output_dict['TRAVELER_ID'])

TRAVELER_ID    0
dtype: int64


In [1667]:
get_list = ['TRAVEL_ID', 'LOTNO_ADDR','LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD','REVISIT_YN','DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION']

tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[1]+extender))
tmp_df = tmp_df.set_index(get_list[0])
output_dict['방문지정보'] = tmp_df[get_list[1:]]
print_isna(output_dict['방문지정보'])
# print(tmp_df)

LOTNO_ADDR             8381
LODGING_TYPE_CD       31832
VISIT_AREA_TYPE_CD        0
REVISIT_YN             6823
DGSTFN                 6839
REVISIT_INTENTION      6839
RCMDTN_INTENTION       6839
dtype: int64


C:\Users\jinmi\AppData\Local\Temp\ipykernel_17848\775514447.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[1]+extender))


In [1668]:
for keys in input_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = input_dict[keys]
    print("----------------------------------------")
    print(tmp_df.index)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)


여행객
----------------------------------------
Index(['c_c003867', 'c_c000679', 'c_c013674', 'c_c010826', 'c_c012020',
       'c_c009357', 'c_c012760', 'c_c003756', 'c_c003541', 'c_c002909',
       ...
       'c_c013812', 'c_c012432', 'c_c009023', 'c_c010863', 'c_c001641',
       'c_c005317', 'c_c000381', 'c_c008277', 'c_c012053', 'c_c007519'],
      dtype='object', name='TRAVEL_ID', length=3200)
--------------------------------------------------------------------------------
여행
----------------------------------------
Index(['c_c011995', 'c_c008882', 'c_c003940', 'c_c012638', 'c_c011983',
       'c_c012367', 'c_c013909', 'b_b006673', 'c_c012881', 'c_c002721',
       ...
       'c_c002389', 'c_c003692', 'c_c013085', 'c_c011684', 'c_c000183',
       'c_c012049', 'c_c010871', 'c_c004312', 'c_c009302', 'c_c000104'],
      dtype='object', name='TRAVEL_ID', length=3200)
--------------------------------------------------------------------------------
동반자정보
-------------------------------------

In [1669]:
for keys in input_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = input_dict[keys]
    print("----------------------------------------")
    print(tmp_df.columns)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)
print("----------------------------------------"*2)
print("----------------------------------------"*2)
print("----------------------------------------"*2)
for keys in output_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = output_dict[keys]
    print("----------------------------------------")
    print(tmp_df.columns)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)


여행객
----------------------------------------
Index(['INCOME', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8'],
      dtype='object')
--------------------------------------------------------------------------------
여행
----------------------------------------
Index(['TRAVEL_MISSION'], dtype='object')
--------------------------------------------------------------------------------
동반자정보
----------------------------------------
Index(['REL_CD', 'COMPANION_AGE_GRP'], dtype='object')
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
TRAVELER_ID

# 방분지 정보 정제

> 아래는 해당 번호들 제외하고 걸러야한다

VIS - VISIT_AREA_TYPE_CD 사용할 항목들

            1:자연관광지
            2:역사/유적/종교 시설
            3:문화 시설
            4:상업지구
            5:레저
            6:테마시설
            7:산책로, 둘레길 등
            8:지역 축제/행사
            13: 체험 활동 관광지

            


> 아래는 계산해서 SCORE로 추가

(DGSTFN + REVISIT_INTENTOIN + RCMDTN_INTENTION) / 3 =======>>>>>>>>> 실제 출력
    
    모두 tn_visit_area_info_방문지정보_C 에 존재

    '()' 안의 더해지는 값들은 각 값의 범위가 1~5


> 최종 11352개의 방문지 존재 해당 정보를 바탕으로 데이터 수립 

    TRAVERL_ID에 해당 하는 정보들을 추가 적으로 붙여넣어 준다.



In [1670]:
tmp_df = output_dict['방문지정보']
print(tmp_df.head())

new_df = tmp_df[tmp_df['VISIT_AREA_TYPE_CD'].isin([1,2,3,4,5,6,7,8,13])]
new_df.loc[:,'SCORE'] = ((new_df['DGSTFN'] + new_df['REVISIT_INTENTION'] + new_df['RCMDTN_INTENTION']) / 3.0).round(4)
# print(new_df)
output_dict['방문지정보'] = new_df
# print(output_dict['방문지정보'])

duplicated_index = new_df.index.duplicated()
print(duplicated_index)

                      LOTNO_ADDR  LODGING_TYPE_CD  VISIT_AREA_TYPE_CD  \
TRAVEL_ID                                                               
c_c005733  충북 영동군 영동읍 계산리 653-12              NaN                  11   
c_c000326         전남 여수시 수정동 772              NaN                  11   
c_c003528     충북 진천군 진천읍 읍내리 165              NaN                   8   
c_c001050    전북 전주시 완산구 교동 산 1-3              NaN                   2   
c_c001776  충남 홍성군 서부면 남당리 767-18              NaN                   1   

          REVISIT_YN  DGSTFN  REVISIT_INTENTION  RCMDTN_INTENTION  
TRAVEL_ID                                                          
c_c005733          N     4.0                3.0               3.0  
c_c000326          N     5.0                5.0               5.0  
c_c003528          Y     3.0                4.0               4.0  
c_c001050          N     5.0                5.0               4.0  
c_c001776          N     4.0                4.0               4.0  
[False False

C:\Users\jinmi\AppData\Local\Temp\ipykernel_17848\4273098675.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:,'SCORE'] = ((new_df['DGSTFN'] + new_df['REVISIT_INTENTION'] + new_df['RCMDTN_INTENTION']) / 3.0).round(4)


In [1671]:
# tmp_df = input_dict['방문지정보']
# print(tmp_df.head())

# new_df = tmp_df[tmp_df['VISIT_AREA_TYPE_CD'].isin([1,2,3,4,5,6,7,8,13])]
# # print(new_df)
# new_df = new_df.drop("VISIT_AREA_TYPE_CD", axis='columns')
# input_dict['방문지정보'] = new_df
# # print(output_dict['방문지정보'])

# duplicated_index = new_df.index.duplicated()
# print(duplicated_index)

# 데이터 이어 붙이기

나머지 정보들을 output_dict['방문자정보']에 적절한 client_code에 이어 붙여 준다.


여행객
----------------------------------------
Index(['INCOME', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8'],
      dtype='object')
--------------------------------------------------------------------------------
여행
----------------------------------------
Index(['TRAVEL_MISSION'], dtype='object')
--------------------------------------------------------------------------------
동반자정보
----------------------------------------
Index(['REL_CD', 'COMPANION_AGE_GRP'], dtype='object')
--------------------------------------------------------------------------------
방문지정보
----------------------------------------
Index(['LODGING_TYPE_CD'], dtype='object')
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
TRAVELER_ID
----------------------------------------
Index(['TRAVELER_ID'], dtype='object')
--------------------------------------------------------------------------------
방문지정보
----------------------------------------
Index(['LOTNO_ADDR', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'DGSTFN',
       'REVISIT_INTENTION', 'RCMDTN_INTENTION'],
      dtype='object')
--------------------------------------------------------------------------------

In [1672]:
output_dict['최종'] = output_dict['방문지정보'].copy()

In [1673]:
output_dict['최종']
tmp_df = output_dict['최종']
for col_name in output_dict['최종'].columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")

LOTNO_ADDR
885
LODGING_TYPE_CD
11295
VISIT_AREA_TYPE_CD
0
REVISIT_YN
17
DGSTFN
18
REVISIT_INTENTION
18
RCMDTN_INTENTION
18
SCORE
18


In [1674]:
tmp_df = output_dict['최종']
# tmp_df = tmp_df.merge(input_dict['동반자정보'], left_index=True, right_index=True)
tmp_df = tmp_df.join(input_dict['동반자정보'], how='inner', on=['TRAVEL_ID'])
# tmp_df.loc['a_a000202', 'COMPANION_AGE_GRP'] = -1
# print(tmp_df.loc['a_a000202'])
print(input_dict['동반자정보'].shape)
print(pd.isna(input_dict['동반자정보']).sum())
print(tmp_df)

(4741, 2)
REL_CD               0
COMPANION_AGE_GRP    0
dtype: int64
                         LOTNO_ADDR  LODGING_TYPE_CD  VISIT_AREA_TYPE_CD  \
TRAVEL_ID                                                                  
c_c003528        충북 진천군 진천읍 읍내리 165              NaN                   8   
c_c003528        충북 진천군 진천읍 읍내리 165              NaN                   8   
c_c003528        충북 진천군 진천읍 읍내리 165              NaN                   8   
c_c003528                       NaN              NaN                   1   
c_c003528                       NaN              NaN                   1   
...                             ...              ...                 ...   
c_c006215                       NaN              NaN                   1   
c_c006215                       NaN              NaN                   1   
c_c008905  충남 천안시 동남구 목천읍 남화리 230-1              NaN                   2   
c_c004061         전남 구례군 광의면 방광리 70              NaN                   2   
c_c005594          

In [1675]:
for keys in input_dict:
    tmp_df = input_dict[keys]
    tmp_col = tmp_df.columns
    # for col_name in tmp_col:
    #     tmp_df[col_name]
    
    if keys=="동반자정보":
        break
    
    print(keys)
    print(output_dict['최종'].shape)
    print(output_dict['최종'].join(tmp_df).shape)
    output_dict['최종'] = output_dict['최종'].join(tmp_df)
    print(output_dict['최종'].columns)
    
        

여행객
(11352, 8)
(11352, 21)
Index(['LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN',
       'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'SCORE', 'INCOME',
       'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8'],
      dtype='object')
여행
(11352, 21)
(11352, 22)
Index(['LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN',
       'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'SCORE', 'INCOME',
       'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
       'TRAVEL_MISSION'],
      dtype='object')


In [1676]:
tmp_df = output_dict['최종']
for col_name in output_dict['최종'].columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")
    

tmp_df.to_csv("./data/tmp_dataset.csv")

LOTNO_ADDR
885
LODGING_TYPE_CD
11295
VISIT_AREA_TYPE_CD
0
REVISIT_YN
17
DGSTFN
18
REVISIT_INTENTION
18
RCMDTN_INTENTION
18
SCORE
18
INCOME
2345
HOUSE_INCOME
4581
TRAVEL_MOTIVE_1
2348
TRAVEL_MOTIVE_2
2709
TRAVEL_MOTIVE_3
3128
TRAVEL_STYL_1
2345
TRAVEL_STYL_2
2345
TRAVEL_STYL_3
2345
TRAVEL_STYL_4
2345
TRAVEL_STYL_5
2345
TRAVEL_STYL_6
2345
TRAVEL_STYL_7
2345
TRAVEL_STYL_8
2345
TRAVEL_MISSION
2322


# 데이터 보정

> LOTNO_ADDR

    885

    필수로 존재해야 하는 값이다.
    없으면 해당 행은 모두 제거

=================
> LODGING_TYPE_CD

    11295

    숙소를 쓰는 경우가 흔하지 않은 것인지  na가 많다
    나머지는 모두 0으로 채운다

=================
> VISIT_AREA_TYPE_CD

    0


=================
> REVISIT_YN

    17
    재방문 할 것인지 여부
    yes - 1
    no - 0으로 변경 후 
    max 값으로 나머지 보정

=================
> DGSTFN

    18
    만족도 점수

    필수 값이다
    평균 값으로 대체

=================
> REVISIT_INTENTION
    
    18
    재방문의향

    필수 값이다
    평균 값으로 대체


=================
> RCMDTN_INTENTION

    18
    추천의향이다

    필수 값이다
    평균 값으로 대체

=================
> SCORE

    18
    위의 값으로 계산된 점수 이다

    필수 값이다
    평균 값으로 대체

=================
> INCOME
    
    2345
    개인 소득 수준이다

    평균값으로 대체


=================

> HOUSE_INCOME
    
    4581

    사용x 날림

=================
> TRAVEL_MOTIVE_1

    2348

    

# 동기값들은 모두 0으로 보정
# 귀찮아서 적지 않았거나 다른 이유로든 적지 않았다면 해당 부분도 대상의 성격 중 일부 라고 할 수 있음
# default로 10:기타로 설정

=================
> TRAVEL_MOTIVE_2
    
    2709
=================
> TRAVEL_MOTIVE_3
    
    3128
=================
> TRAVEL_STYL_1
   
    2345

# 여기서부터 1~8까지의 값중 MAX로 하여 새로운 열을 생성
# default로 4:중립 사용

=================
> TRAVEL_STYL_2
    
    2345
=================
> TRAVEL_STYL_3
    
    2345
=================
> TRAVEL_STYL_4
    
    2345
=================
> TRAVEL_STYL_5
    
    2345
=================
> TRAVEL_STYL_6
    
    2345
=================
> TRAVEL_STYL_7
    
    2345
=================
> TRAVEL_STYL_8
    
    2345
=================
> TRAVEL_MISSION
    
    2322
    
# 해당 값은 0으로 초기화 적당한 default 값이 존재하지 않고
=================

In [1677]:
final_df_bk = tmp_df.copy().reset_index()
final_df = final_df_bk.copy()
print(final_df.index)

RangeIndex(start=0, stop=11352, step=1)


In [1678]:
target_col = "LOTNO_ADDR"

print(final_df.shape)
print_isna(final_df[target_col])
tmp_df = final_df

tmp_df = tmp_df.loc[~pd.isna(tmp_df.loc[:,target_col])]



print(tmp_df.shape)
print_isna(tmp_df[target_col])


(11352, 23)
885
(10467, 23)
0


In [1679]:
final_df = tmp_df

In [1680]:
target_col = "LODGING_TYPE_CD"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df
tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df.shape)
print_isna(tmp_df[target_col])


(10467, 23)
10421
(10467, 23)
0


In [1681]:
final_df = tmp_df

In [1682]:
target_col = "REVISIT_YN"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df.loc[tmp_df[target_col]=="Y", target_col] = 1
tmp_df.loc[tmp_df[target_col]=="N", target_col] = 0

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].max())

print(tmp_df[target_col])

o = ((tmp_df[target_col]==1).sum())
z = ((tmp_df[target_col]==0).sum())

print(o+z)

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(10467, 23)
12
0        1
1        0
2        1
3        0
4        1
        ..
11347    1
11348    0
11349    1
11350    0
11351    0
Name: REVISIT_YN, Length: 10467, dtype: int64
10467
(10467, 23)
0


In [1683]:
final_df = tmp_df

In [1684]:
target_col = "DGSTFN"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(10467, 23)
13
0    4.0
1    4.0
2    2.0
3    4.0
4    4.0
Name: DGSTFN, dtype: float64
(10467, 23)
0


In [1685]:
final_df = tmp_df

In [1686]:
target_col = "REVISIT_INTENTION"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(10467, 23)
13
0    4.0
1    4.0
2    3.0
3    4.0
4    4.0
Name: REVISIT_INTENTION, dtype: float64
(10467, 23)
0


In [1687]:
final_df = tmp_df

In [1688]:
target_col = "RCMDTN_INTENTION"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(10467, 23)
13
0    4.0
1    4.0
2    3.0
3    4.0
4    3.0
Name: RCMDTN_INTENTION, dtype: float64
(10467, 23)
0


In [1689]:
final_df = tmp_df

In [1690]:
target_col = "SCORE"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(10467, 23)
13
0    4.0000
1    4.0000
2    2.6667
3    4.0000
4    3.6667
Name: SCORE, dtype: float64
(10467, 23)
0


In [1691]:
final_df = tmp_df

In [1692]:
target_col = "INCOME"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(10467, 23)
2132
0     4.00228
1     4.00228
2     4.00228
3    12.00000
4    10.00000
Name: INCOME, dtype: float64
(10467, 23)
0


In [1693]:
final_df = tmp_df

In [1694]:
target_col = "HOUSE_INCOME"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df
tmp_df = tmp_df.drop(axis=1, columns=target_col)

print(tmp_df.shape)

(10467, 23)
4202
(10467, 22)


In [1695]:
final_df = tmp_df

In [1696]:
print(final_df.shape)
# print_isna(final_df[target_col])

tmp_df = final_df

for i in range(1,4):
    target_col = "TRAVEL_MOTIVE_" + str(i)

    tmp_df[target_col] = tmp_df[target_col].fillna(10)

    print(tmp_df[target_col].head())

    print((tmp_df[target_col]==10).sum())
    print(tmp_df.shape)
    print_isna(tmp_df[target_col])

(10467, 22)
0    10.0
1    10.0
2    10.0
3     3.0
4     9.0
Name: TRAVEL_MOTIVE_1, dtype: float64
2260
(10467, 22)
0
0    10.0
1    10.0
2    10.0
3     1.0
4     1.0
Name: TRAVEL_MOTIVE_2, dtype: float64
2509
(10467, 22)
0
0    10.0
1    10.0
2    10.0
3    10.0
4     2.0
Name: TRAVEL_MOTIVE_3, dtype: float64
3097
(10467, 22)
0


In [1697]:
final_df = tmp_df

In [1698]:
tmp_col = tmp_df.columns[-9:-1]
print(tmp_df[tmp_col].describe())
print(tmp_col)
print(tmp_df[tmp_col].mode(axis=0))
print(tmp_df[tmp_col].mode(axis=1))
print(tmp_df[tmp_col].mode(axis=1)[0].value_counts())
tmp_df['TRAVEL_STYL_0'] = tmp_df[tmp_col].mode(axis=1)[0]


       TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  TRAVEL_STYL_4  \
count    8335.000000    8335.000000    8335.000000    8335.000000   
mean        2.980324       2.657229       2.530414       3.442112   
std         1.626469       1.660236       1.506328       1.588212   
min         1.000000       1.000000       1.000000       1.000000   
25%         2.000000       1.000000       1.000000       2.000000   
50%         3.000000       2.000000       2.000000       3.000000   
75%         4.000000       4.000000       3.000000       4.000000   
max         7.000000       7.000000       7.000000       7.000000   

       TRAVEL_STYL_5  TRAVEL_STYL_6  TRAVEL_STYL_7  TRAVEL_STYL_8  
count    8335.000000    8335.000000    8335.000000    8335.000000  
mean        3.712537       3.732214       3.781524       4.756569  
std         1.777632       1.659535       1.991798       1.770254  
min         1.000000       1.000000       1.000000       1.000000  
25%         2.000000       2.000000   

         0    1   2   3
0      NaN  NaN NaN NaN
1      NaN  NaN NaN NaN
2      NaN  NaN NaN NaN
3      4.0  NaN NaN NaN
4      3.0  5.0 NaN NaN
...    ...  ...  ..  ..
11347  4.0  NaN NaN NaN
11348  4.0  NaN NaN NaN
11349  4.0  NaN NaN NaN
11350  1.0  4.0 NaN NaN
11351  1.0  4.0 NaN NaN

[10467 rows x 4 columns]
2.0    2687
1.0    2082
4.0    1319
3.0    1091
6.0     556
5.0     449
7.0     151
Name: 0, dtype: int64


In [1699]:
print(final_df.shape)
# print_isna(final_df[target_col])

tmp_df = final_df

for i in range(0,9):
    target_col = "TRAVEL_STYL_" + str(i)

    print((tmp_df[target_col]==4).sum())
    
    tmp_df[target_col] = tmp_df[target_col].fillna(4)

    print(tmp_df[target_col].head())

    print((tmp_df[target_col]==4).sum())
    print(tmp_df.shape)
    print_isna(tmp_df[target_col])

(10467, 23)
1319
0    4.0
1    4.0
2    4.0
3    4.0
4    3.0
Name: TRAVEL_STYL_0, dtype: float64
3451
(10467, 23)
0
1781
0    4.0
1    4.0
2    4.0
3    1.0
4    4.0
Name: TRAVEL_STYL_1, dtype: float64
3913
(10467, 23)
0
1298
0    4.0
1    4.0
2    4.0
3    4.0
4    1.0
Name: TRAVEL_STYL_2, dtype: float64
3430
(10467, 23)
0
1154
0    4.0
1    4.0
2    4.0
3    4.0
4    5.0
Name: TRAVEL_STYL_3, dtype: float64
3286
(10467, 23)
0
1981
0    4.0
1    4.0
2    4.0
3    4.0
4    6.0
Name: TRAVEL_STYL_4, dtype: float64
4113
(10467, 23)
0
1760
0    4.0
1    4.0
2    4.0
3    1.0
4    5.0
Name: TRAVEL_STYL_5, dtype: float64
3892
(10467, 23)
0
2016
0    4.0
1    4.0
2    4.0
3    4.0
4    2.0
Name: TRAVEL_STYL_6, dtype: float64
4148
(10467, 23)
0
918
0    4.0
1    4.0
2    4.0
3    1.0
4    3.0
Name: TRAVEL_STYL_7, dtype: float64
3050
(10467, 23)
0
1402
0    4.0
1    4.0
2    4.0
3    4.0
4    3.0
Name: TRAVEL_STYL_8, dtype: float64
3534
(10467, 23)
0


In [1700]:
tmp_col = tmp_df.columns[-10:-2].tolist()
tmp_col.append(tmp_df.columns[-1])
print(tmp_df[tmp_col].describe())
print(tmp_col)
print(tmp_df[tmp_col].mode(axis=0))
print(tmp_df[tmp_col].mode(axis=1))
print(tmp_df[tmp_col].mode(axis=1)[0].value_counts())


       TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  TRAVEL_STYL_4  \
count   10467.000000   10467.000000   10467.000000   10467.000000   
mean        3.188019       2.930735       2.829751       3.555747   
std         1.508368       1.577139       1.468720       1.434946   
min         1.000000       1.000000       1.000000       1.000000   
25%         2.000000       2.000000       2.000000       2.000000   
50%         4.000000       3.000000       3.000000       4.000000   
75%         4.000000       4.000000       4.000000       4.000000   
max         7.000000       7.000000       7.000000       7.000000   

       TRAVEL_STYL_5  TRAVEL_STYL_6  TRAVEL_STYL_7  TRAVEL_STYL_8  \
count   10467.000000   10467.000000   10467.000000   10467.000000   
mean        3.771090       3.786758       3.826025       4.602465   
std         1.590494       1.484812       1.779563       1.608811   
min         1.000000       1.000000       1.000000       1.000000   
25%         3.000000       3.0000

In [1701]:
tmp_col = tmp_df.columns
print(tmp_col)
new_col = ['TRAVEL_ID', 'LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD',
       'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'SCORE', 'INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_0', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8', 'TRAVEL_MISSION']
tmp_df = tmp_df[new_col]
tmp_col = tmp_df.columns
print(tmp_col)

Index(['TRAVEL_ID', 'LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD',
       'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'SCORE', 'INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8', 'TRAVEL_MISSION', 'TRAVEL_STYL_0'],
      dtype='object')
Index(['TRAVEL_ID', 'LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD',
       'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'SCORE', 'INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_0', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2',
       'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6',
       'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MISSION'],
      dtype='object')


In [1702]:
final_df = tmp_df

In [1703]:
target_col = "TRAVEL_MISSION"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(10467, 23)
2144
0               2;
1               2;
2               2;
3                0
4    21;24;27;4;5;
Name: TRAVEL_MISSION, dtype: object
(10467, 23)
0


In [1704]:
final_df = tmp_df

In [1705]:
tmp_df = final_df
for col_name in final_df.columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")
    

tmp_df.to_csv("./data/final_dataset.csv")

TRAVEL_ID
0
LOTNO_ADDR
0
LODGING_TYPE_CD
0
VISIT_AREA_TYPE_CD
0
REVISIT_YN
0
DGSTFN
0
REVISIT_INTENTION
0
RCMDTN_INTENTION
0
SCORE
0
INCOME
0
TRAVEL_MOTIVE_1
0
TRAVEL_MOTIVE_2
0
TRAVEL_MOTIVE_3
0
TRAVEL_STYL_0
0
TRAVEL_STYL_1
0
TRAVEL_STYL_2
0
TRAVEL_STYL_3
0
TRAVEL_STYL_4
0
TRAVEL_STYL_5
0
TRAVEL_STYL_6
0
TRAVEL_STYL_7
0
TRAVEL_STYL_8
0
TRAVEL_MISSION
0
